### Instructions
The notebook presents the execution of KONG for general graphs. All graphs should be downloaded from [here](https://ls11-www.cs.tu-dortmund.de/staff/morris/graphkerneldatasets) as they are stored in a specific format. Please make sure you have downloaded and decompressed the corresponding graph dataset in the folder **general\_graphs**. Other formats have to be processed such that we obtain the following output:
* Vs - list of hashmaps for the nodes, one for each graph. Each hash table maps the node id to its label. For unlabeled graphs this can be set to '1'.
* Es - list of hashmaps for the edges, one for each graph. For each node u, we store a list of u's neighbors. (The list is assumed to respect the order of the nodes, if order information is provided.)
* classes - a list with the class per graph.
* set\_labels - a set of the labels that appear in all graphs.

Before running the code please download 10 files with random numbers from [here](https://web.archive.org/web/20160119150146/http://stat.fsu.edu/pub/diehard/cdrom/) and store them into the folder **random** (if there is no such folder, create it). Also, create the folders **feature maps** and **graphs**.

In [1]:
import read_write_utilities

In [2]:
dirname = 'general_graphs/'
filenames = ['MUTAG', 'ENZYMES','PROTEINS', 'NCI1', 'PTC_FM', 'MSRC_9', 'BZR', 'COX2', 'DHFR', 'NCI109']
filename = filenames[5]
print(filename)

MSRC_9


In [3]:
#read the graphs into the corresponding data structures 
Vs, Es, classes, set_labels = read_write_utilities.read_standard_graph(dirname, filename)
print(len(Vs), len(Es))

general_graphs//MSRC_9/MSRC_9_A.txt
222 222


In [4]:
h = 3 # the max subtree depth for each node
max_p = 2 # the max polynomial degree, here it is only for scalability 
table_size = 500 # the feature dimensionality for Tensorsketch 
max_value = 100000 # the total number of features that we will have to sketch
nr_tables = 1 # just one table for Tensorsketch, read the original paper for details: http://www.itu.dk/people/pagh/papers/tensorsketch.pdf
relabel = False # read the KONG paper for details on relabeling

In [5]:
import subtree_kernels
from count_sketch import CountSketch
import tensorsketch

In [6]:
random_files = 'random/'
cs = CountSketch(table_size, nr_tables*max_p, random_files, max_value)
cs_cosine = CountSketch(table_size, nr_tables*max_p, random_files, max_value)     

Count-Sketch data structure initialized
Count-Sketch data structure initialized


In [7]:
vectors, vectors_cosine = subtree_kernels.graph2map(Vs, Es, len(classes) + 1, set_labels, h, relabel,  cs, cs_cosine, nr_tables, max_p)
WL_feature_maps = subtree_kernels.graph2WLmap(Vs, Es, len(classes) + 1, set_labels, h)

222 222 222
100
10
200
10
100
200


In [8]:
outputpath = 'feature_maps/' + filename + '_'
relabeled = ''
if relabel:
    relabeled = '_relabeled'
for k in range(1, h+1):
    for p in range(1, max_p+1):
        output = outputpath + str(k) + '_' + str(p) + relabeled + '.txt'
        output_cosine = outputpath + str(k) + '_' + str(p) + '_cosine' + relabeled + '.txt'
           
        read_write_utilities.write_vectors_to_file(vectors[(k-1)*max_p + p-1], classes, output)
        read_write_utilities.write_vectors_to_file(vectors_cosine[(k-1)*max_p + p-1], classes, output_cosine)
output_WL = outputpath + str(k) + '_1_WL.txt'
read_write_utilities.write_WL_vectors_to_file(WL_feature_maps, k, classes, output_WL)

In [13]:
#Train Linear SVMs
import train_svm
p = 2
filepath = 'feature_maps/'  + filename + '_' + str(h) + '_' + str(p) + relabeled + '.txt'
X_kong, y_kong = train_svm.read_graphs(filepath)
X_kong = X_kong.astype(float)
print(X_kong.shape)

(221, 68)


In [10]:
#10-fold accuracy validation
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
print('Results for KONG')
nr = 10
for i in range(nr):
    np.random.seed(i)
    perm = np.random.permutation(X_kong.shape[0])
    X_kong = X_kong[perm]
    y_kong = y_kong[perm]
    libsvm_clf = LinearSVC(penalty='l1', C= 1.0, loss='squared_hinge', dual=False, tol=1e-3)
    k_fold = KFold(n_splits=10)
    print(i, np.mean(cross_val_score(libsvm_clf, X_kong, y_kong, cv=k_fold)))

Results for KONG
0 0.9456521739130433
1 0.9545454545454547
2 0.9594861660079053
3 0.9500000000000002
4 0.9547430830039525
5 0.95
6 0.9545454545454545
7 0.9590909090909092
8 0.9545454545454547
9 0.9454545454545455


In [11]:
filepath = 'feature_maps/'  + filename + '_' + str(h) + '_' + str(1)  + '_WL.txt'
X_wl, y_wl = train_svm.read_graphs(filepath)
X_wl = X_wl.astype(float)
print(X_wl.shape)

(221, 12768)


In [12]:
print('Results for WL')
nr = 10
for i in range(nr):
    np.random.seed(i)
    perm = np.random.permutation(X_wl.shape[0])
    X_wl = X_wl[perm]
    y_wl = y_wl[perm]
    libsvm_clf = LinearSVC(penalty='l1', C= 1.0, loss='squared_hinge', dual=False, tol=1e-3)
    k_fold = KFold(n_splits=10)
    print(i, np.mean(cross_val_score(libsvm_clf, X_wl, y_wl, cv=k_fold)))

Results for WL
0 0.9183794466403162
1 0.900790513833992
2 0.9094861660079052
3 0.9138339920948617
4 0.9181818181818182
5 0.9047430830039527
6 0.9049407114624506
7 0.9136363636363637
8 0.9096837944664034
9 0.9136363636363637
